In [184]:
from nltk.stem.snowball import SnowballStemmer
import re
import math
import json

# Implementação de uma classe auxiliar para fazer stemming
class Stemmer:
  def __init__(self):
    self.stemmer = SnowballStemmer('portuguese')

  def stem(self, text):
    # Utiliza uma expressão regular para substituir caracteres
    # especiais por espaços
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    
    # Remove espaços extras no início e no final
    text = text.strip()
    
    text = text.split()
    return [self.stemmer.stem(word) for word in text]


# Implementação do indexador BM25 para documentos
class BM25Indexer:
  stemmer = Stemmer()

  def __init__(self, corpus, k1=1.2, b=0.75):
    # Inicializa o indexador BM25 com o corpus de documentos
    # e parâmetros k1 e b. k1 e b são parâmetros de ajuste e geralmente
    # são definidos como k1 = 1.2 e b = 0.75 (pelo Elasticsearch,
    # por exemplo)
    self.corpus = corpus # corpus de documentos
    self.k1 = k1 # ajuda a determinar as características de saturação
    self.b = b # é um multiplicador da razão entre comprimentos
    self.doc_lengths = {}
    for doc in corpus:
      self.doc_lengths[doc['id']] = len(self.tokenize(doc['doc']))
    self.avg_doc_length = sum(self.doc_lengths.values()) / len(self.doc_lengths)
    self.doc_term_freqs = {}
    for doc in corpus:
      self.doc_term_freqs[doc['id']] = self.get_doc_term_freqs(doc)
    self.total_docs = len(corpus)
    self.doc_freqs = self.get_doc_freqs()

  def get_doc_term_freqs(self, doc):
    # Calcula as frequências de termos para cada campo em um documento
    term_freqs = {}
    terms = self.tokenize(doc['doc'])
    for term in terms:
      term_freqs[term] = term_freqs.get(term, 0) + 1
    return term_freqs

  def get_doc_freqs(self):
    # Calcula as frequências de documentos para cada termo em todos
    # os documentos
    doc_freqs = {}
    for doc in self.corpus:
      terms = set(self.tokenize(doc['doc']))
      for term in terms:
        doc_freqs[term] = doc_freqs.get(term, 0) + 1
    return doc_freqs

  def get_idf(self, term):
    # Calcula o fator IDF (Inverse Document Frequency) para um termo
    doc_freq = self.doc_freqs.get(term, 0)
    return math.log(
      (self.total_docs - doc_freq + 0.5) / (doc_freq + 0.5) + 1.0
    )

  def get_bm25_score(self, query, doc):
    # Calcula o escore BM25 para um documento em relação a uma consulta
    score = 0
    terms = self.tokenize(query)
    for term in terms:
      idf = self.get_idf(term)
      if term not in self.doc_term_freqs[doc['id']]:
        continue
      doc_freq = self.doc_term_freqs[doc['id']].get(term, 0)
      doc_length = self.doc_lengths[doc['id']]
      numerator = doc_freq * (self.k1 + 1)
      denominator = (doc_freq + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length))
      score += idf * numerator / denominator
    return score

  def tokenize(self, text):
    # Implementa a lógica de tokenização
    # Vamos usar o Snowball Stemmer para o Português
    return self.stemmer.stem(text)

# Consulta
def search(self, query, fields, top_n=10):
  # Implementa a lógica de busca
  scores = {}
  for doc in self.corpus:
    score = self.get_bm25_score(query, doc)
    scores[doc['id']] = score
  print(scores)
  return scores[:top_n]

In [185]:
# instanciando UM indexador BM25 para cada grupo de teses
thesis = [f'group{i}_thesis.json' for i in range(10)]

bm25_indexer = {}
for file in thesis:
  with open(file, 'r', encoding='utf-8') as f:
    corpus = json.load(f)
    print(f"Total de documentos em {file}: {len(corpus)}")
  bm25_indexer[file] = {
    'title': BM25Indexer([{'id': doc['id'], 'doc': doc['title']} for doc in corpus]),
    'abstract': BM25Indexer([{'id': doc['id'], 'doc': doc['abstract']} for doc in corpus]),
    'keywords': BM25Indexer([{'id': doc['id'], 'doc': doc['keywords']} for doc in corpus])
  }

Total de documentos em group0_thesis.json: 277
Total de documentos em group1_thesis.json: 277
Total de documentos em group2_thesis.json: 277
Total de documentos em group3_thesis.json: 277
Total de documentos em group4_thesis.json: 277
Total de documentos em group5_thesis.json: 277
Total de documentos em group6_thesis.json: 277
Total de documentos em group7_thesis.json: 276
Total de documentos em group8_thesis.json: 276
Total de documentos em group9_thesis.json: 276


In [186]:
# instanciando UM indexador BM25 para cada grupo de teses
thesis = [f'group{i}_thesis.json' for i in range(10)]

# une todos os grupos de teses em uma unica lista
total_thesis = []
for file in thesis:
    with open(file, 'r', encoding='utf-8') as f:
        corpus = json.load(f)
        total_thesis += corpus
        print(f"Total de documentos em {file}: {len(corpus)}")

print(f"Total de documentos em todos os grupos: {len(total_thesis)}")

total_bm25_indexer = {
    'title': BM25Indexer([{'id': doc['id'], 'doc': doc['title']} for doc in total_thesis]),
    'abstract': BM25Indexer([{'id': doc['id'], 'doc': doc['abstract']} for doc in total_thesis]),
    'keywords': BM25Indexer([{'id': doc['id'], 'doc': doc['keywords']} for doc in total_thesis])
}

Total de documentos em group0_thesis.json: 277
Total de documentos em group1_thesis.json: 277
Total de documentos em group2_thesis.json: 277
Total de documentos em group3_thesis.json: 277
Total de documentos em group4_thesis.json: 277
Total de documentos em group5_thesis.json: 277
Total de documentos em group6_thesis.json: 277
Total de documentos em group7_thesis.json: 276
Total de documentos em group8_thesis.json: 276
Total de documentos em group9_thesis.json: 276
Total de documentos em todos os grupos: 2767


In [187]:
# Instanciando um indexador BM25 hibrido com IDF total mas com campos separados
hybrid_bm25_indexer = bm25_indexer.copy()

for file in hybrid_bm25_indexer:
    hybrid_bm25_indexer[file]['title'].doc_freqs = total_bm25_indexer['title'].doc_freqs
    hybrid_bm25_indexer[file]['abstract'].doc_freqs = total_bm25_indexer['abstract'].doc_freqs
    hybrid_bm25_indexer[file]['keywords'].doc_freqs = total_bm25_indexer['keywords'].doc_freqs
    hybrid_bm25_indexer[file]['title'].total_docs = total_bm25_indexer['title'].total_docs
    hybrid_bm25_indexer[file]['abstract'].total_docs = total_bm25_indexer['abstract'].total_docs
    hybrid_bm25_indexer[file]['keywords'].total_docs = total_bm25_indexer['keywords'].total_docs

In [190]:
# Montar uma dataframe pandas com os resultados de busca
# Nós temos 3 cenários:
# 1. Indexador BM25 com documentos separados em 10 grupos (sistemas distribuídos)
# 2. Indexador BM25 com documentos em um único grupo (sistema centralizado)
# 3. Indexador BM25 com documentos separados em 10 grupos mas com IDF total
#    (sistemas distribuídos com IDF total)
#
# Agora precisamos executar os testes (100 consultas) para cada cenário e montar
# uma tabela com os resultados:
# A tabela deve ter 7 colunas:
# 1. Query
# 2. Resultados do cenário 1 (Array dos top-k documentos indexados)
# 3. Resultados do cenário 2 (Array dos top-k documentos indexados)
# 4. Resultados do cenário 3 (Array dos top-k documentos indexados)
# 5. Distancia de Levenshtein entre os resultados do cenário 1 e 2
# 6. Distancia de Levenshtein entre os resultados do cenário 2 e 3
# 7. Distancia de Levenshtein entre os resultados do cenário 1 e 3

# Tradução de função de score
def _get_bm25_score(indexer, query, doc):
  return indexer.get_bm25_score(query, doc)

# Implementação da distância de Levenshtein
def levenshtein_distance(s1, s2):
  print(s1, s2)
  if len(s1) < len(s2):
    return levenshtein_distance(s2, s1)
  if len(s2) == 0:
    return len(s1)
  previous_row = range(len(s2) + 1)
  for i, c1 in enumerate(s1):
    current_row = [i + 1]
    for j, c2 in enumerate(s2):
      insertions = previous_row[j + 1] + 1
      deletions = current_row[j] + 1
      substitutions = previous_row[j] + (c1 != c2)
      current_row.append(min(insertions, deletions, substitutions))
    previous_row = current_row
  return previous_row[-1]

# Implementação da busca para cenário distribuído
def distributed_search(indexer, query, top_n=10):
  # Implementa a lógica de busca
  scores = {}
  for file in indexer:
    for field in indexer[file]:
      for doc in indexer[file][field].corpus:
        score = _get_bm25_score(indexer[file][field], query, doc)
        scores[doc['id']] = score
  # Ordena os documentos por score
  scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
  return scores[:top_n]

# Implementação da busca para cenário centralizado
def centralized_search(indexer, query, top_n=10):
  # Implementa a lógica de busca
  scores = {}
  for field in indexer:
    for doc in indexer[field].corpus:
      score = _get_bm25_score(indexer[field], query, doc)
      scores[doc['id']] = score
  # Ordena os documentos por score
  scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
  return scores[:top_n]

In [191]:
# Criação da tabela de resultados
import pandas as pd

df = pd.DataFrame(columns=['query', 'distributed', 'centralized', 'hybrid', 'd-c', 'c-h', 'd-h'])

# Queries
queries = [
  'sistemas distribuídos',
  'sistemas centralizados',
  'sistemas distribuídos e centralizados',
  'sistemas distribuídos e centralizados e paralelos',
  'sistemas distribuídos e centralizados e paralelos e distribuídos',
  'medicina',
  'medicina e saúde',
  'medicina ou saúde',
  'medicina e saúde e doenças',
  'medicina ou saúde ou doenças',
  'medicina primária',
  'medicina primária e secundária',
  'medicina preventiva',
  'medicamentos',
  'medicamentos e vacinas',
  'vacinas',
  'cardiologia',
  'cardiologia e coração',
  'doença cardíaca',
  'doença coração',
  'tuberculose',
  'tuberculose e doença',
  'tuberculose e doença e pulmonar',
  'tuberculose e doença e pulmonar e respiratória',
  'tuberculose e diabetes',
  'tuberculose e cardiopatia',
  'mecanica',
  'quantica',
  'mecanica quantica',
  'mecanica quantica e relatividade',
  'literatura',
  'literatura e portugues',
  'literatura e portugues e brasileira',
  'povos indigenas',
  'povos indigenas e brasil',
  'povos indigenas e brasil e amazonia',
  'povos indigenas e brasil e amazonia e cultura',
  'cultura',
  'cultura e arte',
  'cultura e arte e musica',
  'cultura e arte e musica e brasil',
  'cultura e arte e musica e brasil e samba',
  'cultura e arte e musica e brasil e samba e carnaval',
  'cultura e arte e musica e brasil e samba e carnaval e rio de janeiro',
  'urbanismo',
  'urbanismo e arquitetura',
  'urbanismo e arquitetura e cidade',
  'urbanismo e arquitetura e cidade e sao paulo',
  'tecnologia',
  'tecnologia e computacao',
  'biofisica',
  'biofisica e fisica',
  'biofisica e fisica e biologia',
  'quimica inorganica',
  'quimica industrial',
  'quimica organica',
]


# Execução dos testes
for query in queries:
  print(f"Executando busca para a query: {query}")
  distributed_results = distributed_search(bm25_indexer, query)
  centralized_results = centralized_search(total_bm25_indexer, query)
  hybrid_results = distributed_search(hybrid_bm25_indexer, query)
  df.loc[len(df)] = [
    query,
    distributed_results,
    centralized_results,
    hybrid_results,
    # Distância de Levenshtein entre os resultados do cenário 1 e 2
    # Os resultados são tuplas (id, score), então precisamos converter
    # para uma lista de ids
    levenshtein_distance([doc[0] for doc in distributed_results], [doc[0] for doc in centralized_results]),
    # Distância de Levenshtein entre os resultados do cenário 2 e 3
    levenshtein_distance([doc[0] for doc in centralized_results], [doc[0] for doc in hybrid_results]),
    # Distância de Levenshtein entre os resultados do cenário 1 e 3
    levenshtein_distance([doc[0] for doc in distributed_results], [doc[0] for doc in hybrid_results]),
  ]

# Exibição da tabela de resultados
df


Executando busca para a query: sistemas distribuídos
[2662, 398, 15, 2737, 2606, 1577, 2115, 1163, 291, 248] [2662, 398, 15, 2737, 1577, 2606, 2115, 1163, 580, 291]
[2662, 398, 15, 2737, 1577, 2606, 2115, 1163, 580, 291] [2662, 398, 15, 2737, 2606, 1577, 2115, 1163, 291, 248]
[2662, 398, 15, 2737, 2606, 1577, 2115, 1163, 291, 248] [2662, 398, 15, 2737, 2606, 1577, 2115, 1163, 291, 248]
Executando busca para a query: sistemas centralizados
[1239, 1247, 1622, 1307, 2671, 2641, 2553, 2717, 1263, 1903] [1239, 1622, 1247, 1307, 2671, 2641, 2553, 2717, 1263, 1903]
[1239, 1622, 1247, 1307, 2671, 2641, 2553, 2717, 1263, 1903] [1239, 1247, 1622, 1307, 2671, 2641, 2553, 2717, 1263, 1903]
[1239, 1247, 1622, 1307, 2671, 2641, 2553, 2717, 1263, 1903] [1239, 1247, 1622, 1307, 2671, 2641, 2553, 2717, 1263, 1903]
Executando busca para a query: sistemas distribuídos e centralizados
[2662, 398, 15, 2737, 1418, 2344, 2641, 2137, 2717, 1243] [2662, 398, 15, 2737, 1418, 2344, 2137, 2641, 2717, 1243]
[2662,

,query,distributed,centralized,hybrid,d-c,c-h,d-h
0,sistemas distribuídos,"[(2662, 8.131716811561223), (398, 6.7883620840...","[(2662, 8.23101548293035), (398, 6.74732807057...","[(2662, 8.131716811561223), (398, 6.7883620840...",4,4,0
1,sistemas centralizados,"[(1239, 4.184239326778684), (1247, 4.019544160...","[(1239, 4.173932559554144), (1622, 4.034509598...","[(1239, 4.184239326778684), (1247, 4.019544160...",2,2,0
2,sistemas distribuídos e centralizados,"[(2662, 10.243753898976703), (398, 6.788362084...","[(2662, 10.368843246721145), (398, 6.747328070...","[(2662, 10.243753898976703), (398, 6.788362084...",2,2,0
3,sistemas distribuídos e centralizados e paralelos,"[(2662, 12.355790986392183), (591, 11.49923258...","[(2662, 12.50667101051194), (591, 11.256183199...","[(2662, 12.355790986392183), (591, 11.49923258...",2,2,0
4,sistemas distribuídos e centralizados e parale...,"[(2662, 20.301700001710245), (591, 16.33679819...","[(2662, 20.549609746161543), (591, 15.99150134...","[(2662, 20.301700001710245), (591, 16.33679819...",0,0,0
5,medicina,"[(1880, 5.948652212207685), (2349, 5.898317508...","[(1880, 6.1093573020219685), (2349, 5.86463552...","[(1880, 5.948652212207685), (2349, 5.898317508...",0,0,0
6,medicina e saúde,"[(2566, 8.149865681926565), (1637, 8.094364098...","[(2566, 8.125366141662132), (1637, 8.125366141...","[(2566, 8.149865681926565), (1637, 8.094364098...",4,4,0
7,medicina ou saúde,"[(2566, 8.149865681926565), (1637, 8.094364098...","[(2566, 8.125366141662132), (1637, 8.125366141...","[(2566, 8.149865681926565), (1637, 8.094364098...",4,4,0
8,medicina e saúde e doenças,"[(2531, 13.476417783403926), (1732, 10.4815732...","[(2531, 13.231085284228067), (1732, 10.5863212...","[(2531, 13.476417783403926), (1732, 10.4815732...",2,2,0
9,medicina ou saúde ou doenças,"[(1732, 10.481573277581536), (2531, 9.12148249...","[(1732, 10.586321209964993), (2531, 8.95542975...","[(1732, 10.481573277581536), (2531, 9.12148249...",3,3,0
